### Problem 1: Structural Sub Type
* Complete the definition of MyClass
* DO NOT USE "Any" in anywhere in your code.

In [1]:
object Problem1{
    class MyClass[A,B,C,D,E,F]() {
        type Func1 = { val a: A } => { val b: B }
        type Func2 = { val b: B } => { val a: A }
        type Func3 = { val c: C } => { val a: A }
        type Func4 = { val f: F } => { val d: D }

        type Ty1 = {
            def apply: { val func: Func1 ; val c: C } => { val b: B ; val d: D }
            def function1: { val func: Func3 } => { val c: C }
            val a: A
            val b: B
            val d: D
        }

        type Ty2 = {
            def apply: { val func: Func2 ; val e: E } => { val b: B ; val f: F ; val c: C }
            def function1: { val func: Func4 } => { val c: C; val e:E }
            val a: A
            val c: C
            val d: D
        }

        /*
        Find suitable common supertype of Ty1 and Ty2,
        and replace "Any" with that type.
        */
        type CommonTy = { val a: A; val d: D }
    }
}

defined object Problem1

In [2]:
val mc = new Problem1.MyClass[Int, Int, Int, Int, Int, Int]()
typeOf[mc.Ty1]<:<typeOf[mc.CommonTy]
!(typeOf[Any]<:<typeOf[mc.CommonTy])

val mc1 = new Problem1.MyClass[Int, Char, String, List[Int], List[Long], List[Char]]()
typeOf[mc1.Ty1]<:<typeOf[mc1.CommonTy]
!(typeOf[Any]<:<typeOf[mc1.CommonTy])

mc: Problem1.MyClass[Int, Int, Int, Int, Int, Int] = ammonite.$sess.cmd0$Helper$Problem1$MyClass@554a689b
res1_1: Boolean = true
res1_2: Boolean = true
mc1: Problem1.MyClass[Int, Char, String, List[Int], List[Long], List[Char]] = ammonite.$sess.cmd0$Helper$Problem1$MyClass@2394020b
res1_4: Boolean = true
res1_5: Boolean = true

### Exercise 2: Regular Expressions
* Regular expression is a sequence of character that defines a pattern.
* In this exercise, we will cover two problems:
  1. Converts a string to regular expression defined below.
  2. Check if a regular expression matches given string.
* For more information on regular expressions, refer to the link below.
  * https://en.wikipedia.org/wiki/Regular_expression
* (*) Syntax of Exp (see Data.scala)
>    E -> a      (single alphanumeric character)
>       | (EE)   (concatenation)
>       | (E|E)  (or)
>       | (E)*   (repetition / kleene star)

#### Exercise 2-1: Regex Converter
* Implement a function that converts a list of characters into a regular expression (Exp).
* The result of the conversion should be a regular expression (Exp) that means the same pattern as the input.
* To solve this problem, implement the two functions described below:
(1) Function that convert list of character to list of token (lexer)
(2) Function that convert list of token to regular expression (parser)
* (Tip) About Lexer
  * Lexer is a function that divides a string into appropriate units, and the unit is called token.
  * Token is defined in Data.scala, but there is no content yet.
  * It is recommended to start by defining an appropriate Token to divide the String into.
  * Information about lexers and tokens can be obtained from the link below. https://en.wikipedia.org/wiki/Lexical_analysis
  * ex)
```
      converter({'(', '(', '2', '3', ')', ')', '*'})
        => EStar( EConcat( EChar('2') , EChar('3') ) )
      converter({'(', '1', '(', '(', '(', '2', '3', ')', ')', '*', '4', ')', ')'})
        => EConcat( EChar('1') , EConcat( EStar( EConcat( EChar('2') , EChar('3') ) ) , EChar('4') ) ) )
```
* (caution)
1. Only '0' ~ '9', 'a' ~ 'z', 'A' ~ 'Z', '|', '*', '(', and ')' will be given as an input.
2. Converter must be able to handle gramatical errors of input.
* Note that the result of converting an erroneous character list should be Error itself, rather than an expression containing Error.
* ex)
```
       (ab)* => EError    [((ab))* is right expression]
       (((34) => EError
```
* (*) "List" is an algebraic datatype which is defined as below.
      sealed abstract class List[A]
      case object Nil extends List[A]
      case class Cons(hd: A, tl: List[A]) extends List[A]
* Scala provides a special syntax for List.
    - "case object" can drop out "()" after the name of case class (like "Nil()")
    - "hd :: tl" means "Cons(hd, tl)".
* For usage, see below "drop_two".
      def drop_two(l: List[Int]): List[Int] =
        l match {
          case hd1::tl1 =>
            tl1 match {
              case hd2::tl2 => tl2
              case Nil => Nil
            }
          case _ => Nil
* ref) Technically, Scala can use special symbols as a valid identifier, so it defines "case class ::" instead "case class Cons".
        Therefore, you can't match List like "case Cons(hd, tl) => ..."

In [3]:
sealed abstract class Token
case class Letter(c: Char) extends Token
case object Bar extends Token
case object Star extends Token
case object Open extends Token
case object Close extends Token
case object Other extends Token

sealed abstract class Exp
case class EChar(c: Char) extends Exp
case class EStar(e: Exp) extends Exp
case class EConcat(e1: Exp, e2: Exp) extends Exp
case class EOr(e1:Exp, e2:Exp) extends Exp
case object EEpsilon extends Exp
case object EError extends Exp

defined class Token
defined class Letter
defined object Bar
defined object Star
defined object Open
defined object Close
defined object Other
defined class Exp
defined class EChar
defined class EStar
defined class EConcat
defined class EOr
defined object EEpsilon
defined object EError

In [4]:
import scala.util.control.TailCalls._
import scala.annotation.tailrec

def lexer(l: List[Char]): List[Token] = {
    def lexerIter(xs: List[Char]): List[Token] = {
        xs match {
            case Nil => Nil
            case hd::tl => {
                if (hd == '(') Open::lexerIter(tl)
                else if (hd == ')') Close::lexerIter(tl)
                else if (hd == '*') Star::lexerIter(tl)
                else if (hd == '|') Bar::lexerIter(tl)
                else if (hd >= '0' && hd <= '9') Letter(hd)::lexerIter(tl)
                else if (hd >= 'a' && hd <= 'z') Letter(hd)::lexerIter(tl)
                else if (hd >= 'A' && hd <= 'Z') Letter(hd)::lexerIter(tl)
                else Other::lexerIter(tl)
            }
        }
    }
    
    @tailrec
    def lexerCont(xs: List[Char], cont: List[Token]=>TailRec[List[Token]]): List[Token] = {
        xs match {
            case Nil => cont(Nil).result
            case hd::tl => {
                if (hd == '(') lexerCont(tl, (r) => tailcall(cont(Open::r)))
                else if (hd == ')') lexerCont(tl, (r) => tailcall(cont(Close::r)))
                else if (hd == '*') lexerCont(tl, (r) => tailcall(cont(Star::r)))
                else if (hd == '|') lexerCont(tl, (r) => tailcall(cont(Bar::r)))
                else if (hd >= '0' && hd <= '9') lexerCont(tl, (r) => tailcall(cont(Letter(hd)::r)))
                else if (hd >= 'a' && hd <= 'z') lexerCont(tl, (r) => tailcall(cont(Letter(hd)::r)))
                else if (hd >= 'A' && hd <= 'Z') lexerCont(tl, (r) => tailcall(cont(Letter(hd)::r)))
                else lexerCont(tl, (r) => tailcall(cont(Other::r)))
            }
        }
    }
//     lexerIter(l)
    lexerCont(l, (r)=>done(r))   
}
lexer("(a(b(cd)))".toList)

import scala.util.control.TailCalls._

import scala.annotation.tailrec


defined function lexer
res3_3: List[Token] = List(
  Open,
  Letter('a'),
  Open,
  Letter('b'),
  Open,
  Letter('c'),
  Letter('d'),
  Close,
  Close,
  Close
)

In [5]:
def parser(l: List[Token]): Exp = {
    def isNotOp(e: Exp): Boolean = {
        e match {
            case EChar('|') => false
            case EChar('(') => false
            case EError => false
            case _ => true
        }
    }
    
    @tailrec
    def parserIter(xs: List[Token], stack: List[Exp]): Exp = {
        xs match {
            case hd::tl => hd match {
                // (single alphanumeric character)
                case Letter(c) => parserIter(tl, EChar(c)::stack)
                case Bar => parserIter(tl, EChar('|')::stack)
                case Open => parserIter(tl, EChar('(')::stack)
                case Close => tl match {
                    // (repetition / kleene star) & tl updated
                    case Star::tl => {
                        stack match {
                            case hd1::tl1 if isNotOp(hd1) => tl1 match {
                                case hd2::tl2 if hd2 == EChar('(') => {
                                    parserIter(tl, EStar(hd1)::tl2)
                                }
                                case _ => EError
                            }
                            case _ => EError
                        }
                    }
                    case _ => {
                        stack match {
                            case hd1::tl1 if isNotOp(hd1) => tl1 match {
                                // (or)
                                case EChar('|')::tl2 => tl2 match {
                                    case hd3::tl3 if isNotOp(hd3) => tl3 match {
                                        case hd4::tl4 if hd4 == EChar('(') => {
                                            parserIter(tl, EOr(hd3, hd1)::tl4)
                                        }
                                        case _ => EError
                                    }
                                    case _ => EError
                                }
                                // (concatenation)                  
                                case hd2::tl2 if isNotOp(hd2) => tl2 match {
                                    case hd3::tl3 if hd3 == EChar('(') => {
                                        parserIter(tl, EConcat(hd2, hd1)::tl3)
                                    }
                                    case _ => EError
                                }
                                case _ => EError
                            }
                            case _ => EError
                        }
                    }
                }
                case _ => EError
            }                                                 
            case _ => stack match {
                case Nil => EEpsilon
                case hd::tl if tl == Nil => hd
                case _ => EError
            }
        }
    }
    parserIter(l, Nil)
}

def converter(l: List[Char]): Exp = {
    parser(lexer(l))
}

converter("((1|(((2|(3A)))*4)))*".toList)

defined function parser
defined function converter
res4_2: Exp = EStar(
  EOr(
    EChar('1'),
    EConcat(EStar(EOr(EChar('2'), EConcat(EChar('3'), EChar('A')))), EChar('4'))
  )
)

In [6]:
converter("(a(b(cd)))".toList) == EConcat(EChar('a'),EConcat(EChar('b'),EConcat(EChar('c'),EChar('d'))))
converter("(1(((23))*4))".toList) == EConcat(EChar('1'),EConcat(EStar(EConcat(EChar('2'),EChar('3'))),EChar('4')))
converter("((23))*".toList) == EStar(EConcat(EChar('2'),EChar('3')))
converter("(23)*".toList) == EError
converter("(C((bv))*)".toList) == EConcat(EChar('C'),EStar(EConcat(EChar('b'),EChar('v'))))
converter("".toList) == EEpsilon

res5_0: Boolean = true
res5_1: Boolean = true
res5_2: Boolean = true
res5_3: Boolean = true
res5_4: Boolean = true
res5_5: Boolean = true

#### Exercise 2-2: Regex Matcher by Brzozowski derivative
* Implement a function that checks if a given regular expression matches a string.
* Since regular expression means a string pattern, you can determine whether a string belongs to the pattern.
* Use the "Brzozowski derivative" shown in the link below to implement a function that determines whether a string is included in the pattern of a regular expression.
  * e.g.)
```
      matcher(converter("((23))*".toList), "2323232323") = true
      matcher(converter("((23))*".toList), "d") = false
      matcher(converter("(1(((23))*4))".toList), "12323234") = true
```
* https://en.wikipedia.org/wiki/Brzozowski_derivative
* https://www.ccs.neu.edu/home/turon/re-deriv.pdf

In [19]:
def matcher(e: Exp, s: String): Boolean = {
    def helper(x: Exp): Boolean = {
        x match {
            case EError => false
            case EEpsilon => true
            case EStar(_) => true
            case EChar(_) => false
            case EConcat(e1, e2) => helper(e1) & helper(e2)
            case EOr(e1, e2) => helper(e1) | helper(e2)
        }
    }
    
    def derivative(x: Exp, c: Char): Exp = {
        x match {
            case EEpsilon => EError
            case EChar(c1) if c1 == c => EEpsilon
            case EChar(_) => EError
            case EStar(e) => EConcat(derivative(e, c), x)
            case EConcat(e1, e2) => {
                if (helper(e1)) EOr(EConcat(derivative(e1, c), e2), derivative(e2, c))
                else EConcat(derivative(e1, c), e2)
            }
            case EOr(e1, e2) => EOr(derivative(e1, c), derivative(e2, c))
            case _ => EError
        }
    }
    
    @tailrec
    def matcherIter(x: Exp, i: Int): Boolean = {
        if (i >= s.length) helper(x)
        else {
            println(s(i))
            println(derivative(x, s(i)))
            matcherIter(derivative(x, s(i)), i+1)
        }
    }
    
    println(e)
    e match {
        case EError => false
        case _ => matcherIter(e, 0)
    }
}

matcher(converter("(a(b(cd)))".toList), "abcd")

EConcat(EChar(a),EConcat(EChar(b),EConcat(EChar(c),EChar(d))))
a
EConcat(EEpsilon,EConcat(EChar(b),EConcat(EChar(c),EChar(d))))
b
EOr(EConcat(EError,EConcat(EChar(b),EConcat(EChar(c),EChar(d)))),EConcat(EEpsilon,EConcat(EChar(c),EChar(d))))
c
EOr(EConcat(EError,EConcat(EChar(b),EConcat(EChar(c),EChar(d)))),EOr(EConcat(EError,EConcat(EChar(c),EChar(d))),EConcat(EEpsilon,EChar(d))))
d
EOr(EConcat(EError,EConcat(EChar(b),EConcat(EChar(c),EChar(d)))),EOr(EConcat(EError,EConcat(EChar(c),EChar(d))),EOr(EConcat(EError,EChar(d)),EEpsilon)))


defined function matcher
res18_1: Boolean = true

In [17]:
matcher(converter("(a(b(cd)))".toList), "abcd")
matcher(converter("(1(((23))*4))".toList), "14")
matcher(converter("((23))*".toList), "2323232323")
matcher(converter("(C((bv))*)".toList), "C")
matcher(converter("(C((bv))*)".toList), "Cbvbvbv")

EConcat(EChar(a),EConcat(EChar(b),EConcat(EChar(c),EChar(d))))
a
EConcat(EEpsilon,EConcat(EChar(b),EConcat(EChar(c),EChar(d))))
b
EOr(EConcat(EError,EConcat(EChar(b),EConcat(EChar(c),EChar(d)))),EConcat(EEpsilon,EConcat(EChar(c),EChar(d))))
c
EOr(EConcat(EError,EConcat(EChar(b),EConcat(EChar(c),EChar(d)))),EOr(EConcat(EError,EConcat(EChar(c),EChar(d))),EConcat(EEpsilon,EChar(d))))
d
EOr(EConcat(EError,EConcat(EChar(b),EConcat(EChar(c),EChar(d)))),EOr(EConcat(EError,EConcat(EChar(c),EChar(d))),EOr(EConcat(EError,EChar(d)),EEpsilon)))
EConcat(EChar(1),EConcat(EStar(EConcat(EChar(2),EChar(3))),EChar(4)))
1
EConcat(EEpsilon,EConcat(EStar(EConcat(EChar(2),EChar(3))),EChar(4)))
4
EOr(EConcat(EError,EConcat(EStar(EConcat(EChar(2),EChar(3))),EChar(4))),EOr(EConcat(EConcat(EConcat(EError,EChar(3)),EStar(EConcat(EChar(2),EChar(3)))),EChar(4)),EEpsilon))
EStar(EConcat(EChar(2),EChar(3)))
2
EConcat(EConcat(EEpsilon,EChar(3)),EStar(EConcat(EChar(2),EChar(3))))
3
EConcat(EOr(EConcat(EError,EChar(3))

res16_0: Boolean = true
res16_1: Boolean = true
res16_2: Boolean = true
res16_3: Boolean = true
res16_4: Boolean = true